<a href="https://colab.research.google.com/github/Manya123-max/Big-Data-Framework/blob/main/BDF6_RDDs_OPERATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Aim:**
The code aims to demonstrate various RDD operations in PySpark using a Twitter dataset. It showcases how to perform common transformations and actions on data in a distributed computing environment, focusing on analyzing and manipulating textual and user-related information from Twitter data.

**Step 1: Import Libraries**

kagglehub: Unused in this script but typically used for accessing Kaggle datasets.



In [ ]:
# Import Libraries
import kagglehub
from pyspark.sql import SparkSession

SparkSession: To initialize the Spark application and perform transformations and actions on the dataset.

In [ ]:
# Create SparkSession
spark = SparkSession.builder \
    .appName("Twitter Data RDD Operations") \
    .master("local[*]") \
    .getOrCreate()

**Step 2: Load the Twitter Dataset**

Dataset: A CSV file containing Twitter data (e.g., usernames, tweet texts).

Conversion: The DataFrame is converted to an RDD to enable low-level RDD operations.

In [ ]:
# Load Twitter Dataset
file_path = "/content/twitter_dataset.csv"
twitter_df = spark.read.csv(file_path, header=True, inferSchema=True)
twitter_rdd = twitter_df.rdd

**Step 3: Sample Data Display**

Retrieves and prints the first 5 rows, converted into dictionaries for readability.

In [ ]:
# Display Sample Data
print("Sample Data:")
for row in twitter_rdd.take(5):
    # Convert each row to a dictionary and print the values explicitly
    print({col: row[col] for col in row.asDict()})

Sample Data:
{'Tweet_ID': '1', 'Username': 'julie81', 'Text': 'Party least receive say or single. Prevent prevent husband affect. May himself cup style evening protect. Effect another themselves stage perform.', 'Retweets': None, 'Likes': None}
{'Tweet_ID': 'Possible try tax share style television with. Successful much sell development economy effect."', 'Username': '2', 'Text': '25', 'Retweets': None, 'Likes': None}
{'Tweet_ID': '2', 'Username': 'richardhester', 'Text': 'Hotel still Congress may member staff. Media draw buy fly. Identify on another turn minute would.', 'Retweets': None, 'Likes': None}
{'Tweet_ID': 'Local subject way believe which question some message. Own all imagine join agency indicate."', 'Username': '35', 'Text': '29', 'Retweets': None, 'Likes': None}
{'Tweet_ID': '3', 'Username': 'williamsjoseph', 'Text': 'Nice be her debate industry that year. Film where generation push discover partner level.', 'Retweets': None, 'Likes': None}


**Step 4: RDD Operations**

1. Map Operation:
Prepare data for further analysis or aggregation.



In [ ]:
# 1. Map Operation: Extract tweet text and user
mapped_rdd = twitter_rdd.map(lambda row: (row["Username"], row["Text"]))
print("Mapped Data:")
print(mapped_rdd.take(5))

Mapped Data:
[('julie81', 'Party least receive say or single. Prevent prevent husband affect. May himself cup style evening protect. Effect another themselves stage perform.'), ('2', '25'), ('richardhester', 'Hotel still Congress may member staff. Media draw buy fly. Identify on another turn minute would.'), ('35', '29'), ('williamsjoseph', 'Nice be her debate industry that year. Film where generation push discover partner level.')]


2. Filter Operation: Extract specific subsets of data based on textual content.

In [ ]:
# 2. filter: Filter tweets that contain the word "Spark"
filtered_rdd = twitter_rdd.filter(lambda row: row.Text is not None and "Nice" in row.Text)
print("Filter Operation:")
print(filtered_rdd.take(5))

Filter Operation:
[Row(Tweet_ID='3', Username='williamsjoseph', Text='Nice be her debate industry that year. Film where generation push discover partner level.', Retweets=None, Likes=None), Row(Tweet_ID='244', Username='melanie77', Text='Nice not become man follow explain why from.', Retweets=None, Likes=None), Row(Tweet_ID='412', Username='anthonysmith', Text='Nice worker hundred should. Gun light federal ever.', Retweets=None, Likes=None), Row(Tweet_ID='539', Username='sdean', Text='Office learn process fly. Nice want nor world general public kid.', Retweets=None, Likes=None), Row(Tweet_ID='598', Username='carralicia', Text='Nice accept election respond institution population. Return leave state concern. Matter practice plant then method.', Retweets=None, Likes=None)]


3. FlatMap Operation:  Text preprocessing for word-level analysis.

In [ ]:
# 3. flatMap: Flatten tweets into individual words
flatmapped_rdd = twitter_rdd.flatMap(lambda row: row.Text.split())
print("FlatMap Operation:")
print(flatmapped_rdd.take(10))

FlatMap Operation:
['Party', 'least', 'receive', 'say', 'or', 'single.', 'Prevent', 'prevent', 'husband', 'affect.']


4.  Reduce Operation: Aggregate textual data into a unified corpus.

In [ ]:
# 4. reduce: Concatenate all tweet texts into a single string
reduced_result = twitter_rdd.map(lambda row: row.Text if row.Text is not None else "").reduce(lambda x, y: x + " " + y)
print("Reduce Operation:")
print(reduced_result[:100])  # Display the first 100 characters

Reduce Operation:
Party least receive say or single. Prevent prevent husband affect. May himself cup style evening pro


5.  ReduceByKey Operation: Aggregate counts for user activity analysis.

In [ ]:
# 5. reduceByKey: Count occurrences of each user
user_count_rdd = twitter_rdd.map(lambda row: (row.Username, 1)).reduceByKey(lambda x, y: x + y)
print("ReduceByKey Operation:")
print(user_count_rdd.take(5))

ReduceByKey Operation:
[('julie81', 1), ('2', 97), ('richardhester', 1), ('35', 96), ('williamsjoseph', 2)]


 6. GroupByKey Operation: Analyze all tweets by a specific user collectively.

In [ ]:
from typing_extensions import Text
# 6. groupByKey: Group tweets by user
grouped_rdd = twitter_rdd.map(lambda row: (row.Username, row.Text)).groupByKey()
print("GroupByKey Operation:")
print(grouped_rdd.take(5))

GroupByKey Operation:
[('julie81', <pyspark.resultiterable.ResultIterable object at 0x7f5fb2be9f60>), ('2', <pyspark.resultiterable.ResultIterable object at 0x7f5fb2be97b0>), ('richardhester', <pyspark.resultiterable.ResultIterable object at 0x7f5f96532380>), ('35', <pyspark.resultiterable.ResultIterable object at 0x7f5f96533d90>), ('williamsjoseph', <pyspark.resultiterable.ResultIterable object at 0x7f5f965310f0>)]


7. AggregateByKey Operation: Perform combined metrics on grouped data.

In [ ]:
# 7. aggregateByKey: Compute max and count of tweet lengths per user
aggregated_rdd = twitter_rdd.map(lambda row: (row.Username, len(row.Text) if row.Text is not None else 0)) \
    .aggregateByKey((0, 0),  # Initial (max_length, count)
                    lambda acc, value: (max(acc[0], value), acc[1] + 1),  # SeqOp
                    lambda acc1, acc2: (max(acc1[0], acc2[0]), acc1[1] + acc2[1])  # CombOp
                    )
print("AggregateByKey Operation:")
print(aggregated_rdd.take(5))

AggregateByKey Operation:
[('julie81', (146, 1)), ('2', (3, 97)), ('richardhester', (97, 1)), ('35', (2, 96)), ('williamsjoseph', (109, 2))]


8.  Join Operation: Combine data sources for enriched analysis.

In [ ]:
# 8. join: Join tweet data with user counts
user_rdd = twitter_rdd.map(lambda row: (row.Username, row.Tweet_ID))  # Simulated user RDD
joined_rdd = user_count_rdd.join(user_rdd)
print("Join Operation:")
print(joined_rdd.take(5))

Join Operation:
[(None, (6022, 'Nearly money store style may enjoy. Kid discuss blue save. Model another about along.')), (None, (6022, 'Production thousand will to. Natural research land. Bank option one party nation.')), (None, (6022, 'Source police name operation. Decade unit money return street.')), (None, (6022, 'Section site hour book notice spring body everyone. Person performance gun security least mean such.')), (None, (6022, 'Floor system why position. Prevent stock receive blood family money. Past direction they.'))]


9. CoGroup Operation: Combine related datasets while retaining their structure.

In [ ]:
# 9. cogroup: Combine tweets and user counts using cogroup
cogrouped_rdd = user_count_rdd.cogroup(user_rdd)
print("CoGroup Operation:")
print(cogrouped_rdd.take(5))

CoGroup Operation:
[(None, (<pyspark.resultiterable.ResultIterable object at 0x7f5f965b6890>, <pyspark.resultiterable.ResultIterable object at 0x7f5f965b4520>)), ('ramirezmikayla', (<pyspark.resultiterable.ResultIterable object at 0x7f5f965b4c70>, <pyspark.resultiterable.ResultIterable object at 0x7f5f965b6f80>)), ('22', (<pyspark.resultiterable.ResultIterable object at 0x7f5f965b7850>, <pyspark.resultiterable.ResultIterable object at 0x7f5f965b6dd0>)), ('fieldsbrian', (<pyspark.resultiterable.ResultIterable object at 0x7f5f965b6860>, <pyspark.resultiterable.ResultIterable object at 0x7f5f965b4a60>)), ('12', (<pyspark.resultiterable.ResultIterable object at 0x7f5f965b6470>, <pyspark.resultiterable.ResultIterable object at 0x7f5fb3161240>))]


10. SortBy Operation: Order data for visualization or downstream processing.

In [ ]:
# 10. sortBy: Sort tweets alphabetically by text
sorted_rdd = twitter_rdd.sortBy(lambda row: row.Text if row.Text is not None else "")
print("SortBy Operation:")
print(sorted_rdd.take(5))


SortBy Operation:
[Row(Tweet_ID='Nearly money store style may enjoy. Kid discuss blue save. Model another about along.', Username=None, Text=None, Retweets=None, Likes=None), Row(Tweet_ID='Production thousand will to. Natural research land. Bank option one party nation.', Username=None, Text=None, Retweets=None, Likes=None), Row(Tweet_ID='Source police name operation. Decade unit money return street.', Username=None, Text=None, Retweets=None, Likes=None), Row(Tweet_ID='Section site hour book notice spring body everyone. Person performance gun security least mean such.', Username=None, Text=None, Retweets=None, Likes=None), Row(Tweet_ID='Floor system why position. Prevent stock receive blood family money. Past direction they.', Username=None, Text=None, Retweets=None, Likes=None)]


11. Take and Collect Operations:  Data sampling and inspection.

In [ ]:
# 11. take: Take the first 3 tweets
taken = twitter_rdd.take(3)
print("Take Operation:")
print(taken)

Take Operation:
[Row(Tweet_ID='1', Username='julie81', Text='Party least receive say or single. Prevent prevent husband affect. May himself cup style evening protect. Effect another themselves stage perform.', Retweets=None, Likes=None), Row(Tweet_ID='Possible try tax share style television with. Successful much sell development economy effect."', Username='2', Text='25', Retweets=None, Likes=None), Row(Tweet_ID='2', Username='richardhester', Text='Hotel still Congress may member staff. Media draw buy fly. Identify on another turn minute would.', Retweets=None, Likes=None)]


12. Collect Operation: Quickly check the contents of the dataset.

In [ ]:
# 12. collect: Collect all tweet data to the driver
collected_data = twitter_rdd.collect()
print("Collect Operation:")
print(collected_data[:3])  # Display the first 3 rows

Collect Operation:
[Row(Tweet_ID='1', Username='julie81', Text='Party least receive say or single. Prevent prevent husband affect. May himself cup style evening protect. Effect another themselves stage perform.', Retweets=None, Likes=None), Row(Tweet_ID='Possible try tax share style television with. Successful much sell development economy effect."', Username='2', Text='25', Retweets=None, Likes=None), Row(Tweet_ID='2', Username='richardhester', Text='Hotel still Congress may member staff. Media draw buy fly. Identify on another turn minute would.', Retweets=None, Likes=None)]


13. Distinct Operation: Identify unique entities in the dataset.

In [ ]:
# 13. distinct: Get distinct users
distinct_users_rdd = twitter_rdd.map(lambda row: row.Username).distinct()
print("Distinct Operation:")
print(distinct_users_rdd.take(5))

Distinct Operation:
['julie81', '2', 'richardhester', '35', 'williamsjoseph']


 14. TakeOrdered Operation: Efficiently fetch sorted data.

In [ ]:
# 14. takeOrdered: Take the first 5 tweets based on text order
take_ordered = twitter_rdd.takeOrdered(5, key=lambda row: row.Text if row.Text is not None else "")
print("TakeOrdered Operation:")
print(take_ordered)

TakeOrdered Operation:
[Row(Tweet_ID='Nearly money store style may enjoy. Kid discuss blue save. Model another about along.', Username=None, Text=None, Retweets=None, Likes=None), Row(Tweet_ID='Production thousand will to. Natural research land. Bank option one party nation.', Username=None, Text=None, Retweets=None, Likes=None), Row(Tweet_ID='Source police name operation. Decade unit money return street.', Username=None, Text=None, Retweets=None, Likes=None), Row(Tweet_ID='Section site hour book notice spring body everyone. Person performance gun security least mean such.', Username=None, Text=None, Retweets=None, Likes=None), Row(Tweet_ID='Floor system why position. Prevent stock receive blood family money. Past direction they.', Username=None, Text=None, Retweets=None, Likes=None)]


15. Count Operation: Compute dataset size.

In [ ]:
# 15. count: Count the number of tweets
tweet_count = twitter_rdd.count()
print("Count Operation:")
print(tweet_count)

Count Operation:
25775


In [ ]:
# Stop SparkSession
spark.stop()


**Result:**
 This code uses RDD transformations and actions to analyze a Twitter dataset efficiently. It demonstrates a wide range of operations to filter, map, aggregate, and compute insights, offering valuable data for understanding user activity, tweet content, and engagement patterns.